In [1]:
# library to make requests to a web page, and print the responce text
import requests 
import re
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from time import sleep
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import sys

import urllib.parse as urlparse
from urllib.parse import parse_qs

import folium

from IPython.display import display, clear_output

import json

import webbrowser as web# to open urls instantly


In [2]:
#Input your dates and persons
location = 'Kefalonia'
country = 'Greece'

month = '08'

day_in = '10'
day_out = '25'

year = 2023

persons = 3

nights = int(day_out) - int(day_in)
pages_to_load = 0 # main page has been loaded. pages_to_load depicts the appended pages to load. 


In [3]:
# Create a string with the main link
main_link = 'https://www.airbnb.gr/s/{5}--{6}/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights={7}&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query={5}%2C%20{6}&checkin={0}-{1}-{2}&checkout={0}-{1}-{3}&adults={4}'#&place_id=ChIJywtkGTF2X0YRZnedZ9MnDag
None

In [4]:
print(main_link.format(year,month,day_in,day_out,persons,location,country,nights)) # print the page

https://www.airbnb.gr/s/Kefalonia--Greece/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=15&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Kefalonia%2C%20Greece&checkin=2023-08-10&checkout=2023-08-25&adults=3


In [5]:
def pageLoad(link,k):
    i = 0
    while i<10:
        sleep(i*2)       
        try:
            html_pages = []   # create a list with the html code for every page
            #html = requests.get(link.format(year,month,day_in,day_out,persons)) # Get the page (requests)
            
            html = requests.get(link.format(year,month,day_in,day_out,persons,location,country,nights)) # Get the page (requests)
            
            html_pages.append(html) # append the html code to the previous list
            #print(html) 
            # Get the content from the list of pages . html_pages_content is also a list 
            html_pages_content = [html.content for html in html_pages]
            # This is the first element of the list 
            html = html_pages_content[0]
            soup = BeautifulSoup(html, "html.parser")
            rows = soup.find('div', class_= 'gh7uyir giajdwt g14v8520 dir dir-ltr').find_all('meta') 
            print(f'Page {k} loaded')
            #c = []
            #c = coordinates(soup)
            
            break
        except:
            i += 1
            if i < 10:
                print(f'Loading Page {k} was unsuccessful for {i} time(s). Trying loading again' )
            else:
                print(f'FATAL ERROR: Trial No.{i} was unsuccessful. Page cannot be loaded.' )
            sleep(1)
    return rows,soup#,c

# gh7uyir giajdwt g1xypvzw g14v8520 dir dir-ltr
# gh7uyir giajdwt g14v8520 dir dir-ltr
# gh7uyir giajdwt g14v8520 dir dir-ltr
# gh7uyir giajdwt g14v8520 dir dir-ltr

In [6]:
def createLists(rows):
    name_lst = []
    url_lst = []
    for i in range(0,len(rows)):
        if 'itemprop="name"' in str(rows[i]):
            name_lst.append(str(rows[i]))
            name_lst[len(name_lst)-1] = name_lst[len(name_lst)-1].replace('<meta content="','')
            name_lst[len(name_lst)-1] = name_lst[len(name_lst)-1].replace('" itemprop="name"/>','')
        elif 'www.airbnb.gr' in str(rows[i]):
            url_lst.append('https://' + str(rows[i]))
            url_lst[len(url_lst)-1] = url_lst[len(url_lst)-1].replace('<meta content="','')
            url_lst[len(url_lst)-1] = url_lst[len(url_lst)-1].replace('" itemprop="url"/>','')   
    return name_lst,url_lst

In [7]:
def nextPage(soup):
    l = len(soup.find('div', class_= 'c1bp99s5 dir dir-ltr').find_all('a')) - 1
    next_page = str(soup.find('div', class_= 'c1bp99s5 dir dir-ltr').find_all('a')[l])#.find('class', class_='_1bfat5l')
    start = next_page.find('href="')
    end = next_page.find('">')
    next_page = next_page[start+6:end]
    next_page = 'https://www.airbnb.gr'+next_page.replace('amp;','')
    print(next_page)
    return next_page

In [8]:
name_lst = []
url_lst = []
coords = {}

In [9]:
rows,soup = pageLoad(main_link,1)
name_lst, url_lst = createLists(rows)

n = soup.find("span", class_="to8eev7 dir dir-ltr").text
n = n.split(' ')

#Number of houses
j = 0
for i in n:
    try:
        try:
            j = int(i)
        except:
            j = float(i)
            j *= 1000
    except:
        pass
print('Appartments for rent:', int(j))

Page 1 loaded
Appartments for rent: 1000


In [10]:
#pages_to_load = int(j)/20 - 1 # main page has been loaded. MAXIMUM pages to load


In [11]:
for i in range(1,pages_to_load+1):
    next_page = nextPage(soup)
    rows,soup = pageLoad(next_page,i+1)
    name_lst2, url_lst2 = createLists(rows)
    
    for rep in range(len(name_lst2)):
        name_lst.append(name_lst2[rep])
        url_lst.append(url_lst2[rep])

In [12]:
print(f'Length of Names List: {len(name_lst)}, Length of url List: {len(url_lst)}')

Length of Names List: 18, Length of url List: 18


In [13]:
# Load every single url site and extract information

In [14]:
def coordinates(r): # Find Coordinates for every apartment
    p_lat = re.compile(r'"lat":([-0-9.]+),')
    p_lng = re.compile(r'"lng":([-0-9.]+),')
    lat = p_lat.findall(r.text)[0]
    lng = p_lng.findall(r.text)[0]
    return lat,lng

In [15]:
def urlLoad(url):
    html_pages = []   # create a list with the html code for every page
    #html = requests.get(url)#.format(year,month,day_in,day_out,persons)) # Get the page (requests)
    html_pages.append(url) # append the html code to the previous list
    # Get the content from the list of pages . html_pages_content is also a list 
    html_pages_content = [url.content for url in html_pages]
    # This is the first element of the list 
    url = html_pages_content[0]
    soup = BeautifulSoup(url, "html.parser")
  
    return soup

In [16]:
def extractInfo(r,link):
    error = 0
    #visitors
    p_vst = re.compile(r'"detailItems":.+επισκέπτες"},')
    vst = p_vst.findall(r.text)[0].split(':"')[2].split(" ")[0]

    try:
        #beds
        p_beds = re.compile(r'"detailItems":.+ κρεβάτι.+},{"__typename":"BasicListItem","title":"')
        beds = p_beds.findall(r.text)[0].split(' κρεβάτι')[0]
        beds = beds[len(beds)-1]
    except:
        beds = None
        
    #baths
    p_baths = re.compile(r'detailItems".+ μπάνι')
    baths = p_baths.findall(r.text)[0].split(' μπάνι')[0]
    baths = baths[len(baths)-1] 
    if baths == 5:
        baths = 1

    #bedrooms
    p_bedrooms = re.compile(r'επισκέπτες"},{"__typename":"BasicListItem","title":".+ υπνοδωμάτι')
    try:
        bedrooms = p_bedrooms.findall(r.text)[0].split(' υπνοδωμάτι')[0]
        bedrooms = bedrooms[len(bedrooms)-1]
        try:
            bedrooms = int(bedrooms)
        except:
            bedrooms = None
    except:
        bedrooms = None
    
    # Allamenities
    
    i = 1
    p_ame = re.compile(r'"seeAllAmenitiesGroups":.+}],"seeAllAmenitiesButton":{')
    while i <= 5:
        try:
            amenities = p_ame.findall(r.text)
            amenities = amenities[0][24:len(amenities[0])-26]
            break
        except:
            print(f'ERROR No {i}: ',amenities,'. Trying loading again!')
            i += 1
            sleep(i)
            r = requests.get(link)
    if i == 6:
        print('Fatal Error. Page has not been loaded.')
        sleep(2)
        error = 1
    return vst,beds,baths,bedrooms,amenities,error

In [17]:
def jsonFileEdit(file):
    amenitiesGrouplist = []
    amenitiesList = []

    for amenityGroup in json_object: #json_object == file
        if amenityGroup['title'] != 'Δεν περιλαμβάνονται':
            amenitiesGrouplist.append(amenityGroup['title'])

            for amenity in amenityGroup['amenities']:
                if amenity['available'] == True:
                    amenitiesList.append(amenity['title'])      
    return amenitiesGrouplist,amenitiesList

In [18]:
coords = {}
visitors_lst = []
beds_lst = []
baths_lst = []
bedrooms_lst = []

#mountain_view = []
#sea_view = []

#warm_water_lst = []
#needs_lst = []
#hangs_lst = []

rep = 1
pagesNotLoaded = 0

#amenGroups = set()
amenSet = set()

amenities_list = []

for page in url_lst:
    
    amenitiesGroup = []
    amenities = []
    
    # Open url
    html = requests.get(page)

    # Find Coordinates for every apartment
    lat, lng = coordinates(html)
    coords[rep] = [lat,lng]

    # Scrape url
    soup = urlLoad(html)
    
    # Extract Information from url
    vst, bed, bths, bdrooms, amens, errors = extractInfo(html,page)
    
    visitors_lst.append(vst)
    beds_lst.append(bed)
    baths_lst.append(bths)
    bedrooms_lst.append(bdrooms)
    
    try:
        json_object = json.loads(amens)
        amenitiesGroup, amenities = jsonFileEdit(json_object)
        amenities_list.append(amenities)
        amenSet.update(amenities)

        """
        amenGroups.update(amenitiesGroup)
        amenSet.update(amenities)
        
        if 'Ζεστό νερό' in amenities:
            warm_water_lst.append(1)
        else:
            warm_water_lst.append(0)
        if 'Είδη πρώτης ανάγκης' in amenities:
            needs_lst.append(1)
        else:
            needs_lst.append(0)
        if 'Κρεμάστρες' in amenities:
            hangs_lst.append(1)
        else:
            hangs_lst.append(0)
        """
    except:
        print('Something goes wrong')
        pass
        """
        warm_water_lst.append(None)
        needs_lst.append(None)
        hangs_lst.append(None)
        """    
    # Output progress
    clear_output(wait=False)
    perc = rep/len(url_lst)*100
    print(f'{round(perc,2)}%')
    pagesNotLoaded += errors
    if pagesNotLoaded > 0:
        print('Pages not loaded:',pagesNotLoaded)
    else:
        print('All pages have been loaded!')
    
    rep += 1
    """
    if rep == 4:
        print('Loading completed')
        break
    """

100.0%
All pages have been loaded!


In [19]:
amenity_df = pd.DataFrame

In [20]:
#json_object

In [21]:
"""
for amenity in json_object:
    for item in amenity['amenities']:
        #print(item['title'])
        if item['available'] == True:
            amenity_df[item['title']] = np.nan
    break
amenity_df
"""
None

In [22]:
data = pd.DataFrame.from_dict(coords, orient='index',
                       columns=['latitude', 'longitude'])

In [23]:
#convert coordinates to float from strings
data['latitude'] = data['latitude'].astype(float)
data['longitude'] = data['longitude'].astype(float)

In [24]:
data['url'] = url_lst
data['Name'] = name_lst
data['Beds'] = beds_lst
data['Visitors'] = visitors_lst
data['Bathrooms'] = baths_lst
data['Bedrooms'] = bedrooms_lst
#data['Warm_Water'] = warm_water_lst
#data['Needs'] = needs_lst
#data['Hangings'] = hangs_lst

In [25]:
data['Amenities'] = amenities_list

In [26]:
try:
    data['Visitors'] = data['Visitors'].astype(int)
except:
    print('Visitors could not be converted to int')
data = data.replace('ό',1) # private bathroom 
data = data.replace('α',2) # not private bathroom  
data = data.replace('None','')

try:
    data['Bathrooms'] = data['Bathrooms'].astype(int)
except:
    print('Bathrooms could not be converted to int')

try:
    data['Beds'] = data['Beds'].astype(int)
    data['Beds'] = data['Beds'].astype(int)
except:
    print('Beds could not be converted to int')
    
try:
    data['Bedrooms'] = data['Bedrooms'].replace(np.nan,0)
    data['Bedrooms'] = data['Bedrooms'].astype(int)
except:
    print('Bedrooms could not be converted to int')
#try:
#    data['Hangings'] = data['Hangings'].astype(int)
#except:
#    print('Hangings could not be converted to int')

In [27]:
# Correction of url
data.url = data.url.str.split('amp;')
data.url = data.url.apply(lambda x: ''.join(map(str, x)))
#web.open(data.url[50]) # open url instantly

In [28]:
# Dataframe to csv file
#data.to_csv('appartments.csv',index=False)

In [29]:
# Open saved scv file
#data = pd.read_csv('appartments.csv')

In [30]:
m = folium.Map(location=[38.228921, 20.567190],tiles='Stamen Terrain')

for i in range(0,len(data)):
   folium.Marker(
      location=[data.iloc[i]['latitude'], data.iloc[i]['longitude']],
      popup = data.iloc[i]['Name'],
   ).add_to(m)
m

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 1 to 18
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   latitude   18 non-null     float64
 1   longitude  18 non-null     float64
 2   url        18 non-null     object 
 3   Name       18 non-null     object 
 4   Beds       18 non-null     int32  
 5   Visitors   18 non-null     int32  
 6   Bathrooms  18 non-null     int32  
 7   Bedrooms   18 non-null     int32  
 8   Amenities  18 non-null     object 
dtypes: float64(2), int32(4), object(3)
memory usage: 1.1+ KB


In [32]:
test = 5
#web.open(data.url[test]) # open url instantly
print(len(data.Amenities[test]),data.Amenities[test])

22 ['Πιστολάκι μαλλιών', 'Σαμπουάν', 'Πλυντήριο ρούχων', 'Στεγνωτήριο', 'Είδη πρώτης ανάγκης', 'Κρεμάστρες', 'Σίδερο', 'Τηλεόραση', 'Κλιματισμός', 'Θέρμανση', 'Πυροσβεστήρας', 'Κουτί πρώτων βοηθειών', 'Wifi', 'Ειδικός χώρος εργασίας', 'Κουζίνα', 'Ψυγείο', 'Μαγειρικά είδη', 'Ιδιωτική είσοδος', 'Ιδιωτική πίσω αυλή – Με πλήρη περίφραξη', 'Έπιπλα εξωτερικού χώρου', 'Υπαίθρια τραπεζαρία', 'Επιτρέπονται οι διαμονές μεγάλης διάρκειας']


In [33]:
data.Amenities[1]

['Πιστολάκι μαλλιών',
 'Σαμπουάν',
 'Ζεστό νερό',
 'Πλυντήριο ρούχων – στο κατάλυμα: Δωρεάν',
 'Είδη πρώτης ανάγκης',
 'Κρεμάστρες',
 'Κλινοσκεπάσματα',
 'Επιπλέον μαξιλάρια και κουβέρτες',
 'Σίδερο',
 'Τηλεόραση',
 'Κλιματισμός',
 'Θέρμανση',
 'Κουτί πρώτων βοηθειών',
 'Wifi',
 'Κουζίνα',
 'Ψυγείο',
 'Μαγειρικά είδη',
 'Πιάτα και μαχαιροπίρουνα',
 'Κουζίνα',
 'Φούρνος',
 'Καφετιέρα',
 'Ιδιωτική είσοδος',
 'Αίθριο ή μπαλκόνι',
 'Δωρεάν χώρος στάθμευσης στην ιδιοκτησία',
 'Επιτρέπονται κατοικίδια',
 'Επιτρέπεται αποθήκευση αποσκευών',
 'Επιτρέπονται οι διαμονές μεγάλης διάρκειας']

In [34]:
amenList = list(amenSet)
split = pd.DataFrame(data['Amenities'], columns = amenList)
split.head()

,Θέα σε παραλία,Μίξερ,Θέα στη θάλασσα,Κλιματισμός,"HDTV 42"" με Κλασική καλωδιακή τηλεόραση",Πλυντήριο πιάτων,Επιτρέπονται κατοικίδια,Ταψί,Στεγνωτήριο,Καφετιέρα,...,Τζάκι εξωτερικού χώρου,Θέα στον ορίζοντα της πόλης,Αίθριο ή μπαλκόνι: Ιδιωτική παροχή,Στεγνωτήριο: Δωρεάν,"ΤΗΛΕΌΡΑΣΗ 32""",Αιώρα,Μπανιέρα,Check-in χωρίς παρουσία οικοδεσπότη,Πιστολάκι μαλλιών,"HDTV 43"" με Chromecast, Netflix, Κλασική καλωδιακή τηλεόραση"


In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 1 to 18
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   latitude   18 non-null     float64
 1   longitude  18 non-null     float64
 2   url        18 non-null     object 
 3   Name       18 non-null     object 
 4   Beds       18 non-null     int32  
 5   Visitors   18 non-null     int32  
 6   Bathrooms  18 non-null     int32  
 7   Bedrooms   18 non-null     int32  
 8   Amenities  18 non-null     object 
dtypes: float64(2), int32(4), object(3)
memory usage: 1.7+ KB


In [36]:
data = data.append(split)

C:\Users\PEROULIS\AppData\Local\Temp\ipykernel_14720\2213528554.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(split)


In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 1 to 18
Columns: 129 entries, latitude to HDTV 43" με Chromecast, Netflix, Κλασική καλωδιακή τηλεόραση
dtypes: float64(6), object(123)
memory usage: 18.3+ KB


In [38]:
data.head()

,latitude,longitude,url,Name,Beds,Visitors,Bathrooms,Bedrooms,Amenities,Θέα σε παραλία,...,Τζάκι εξωτερικού χώρου,Θέα στον ορίζοντα της πόλης,Αίθριο ή μπαλκόνι: Ιδιωτική παροχή,Στεγνωτήριο: Δωρεάν,"ΤΗΛΕΌΡΑΣΗ 32""",Αιώρα,Μπανιέρα,Check-in χωρίς παρουσία οικοδεσπότη,Πιστολάκι μαλλιών,"HDTV 43"" με Chromecast, Netflix, Κλασική καλωδιακή τηλεόραση"
1,38.12712,20.58069,https://www.airbnb.gr/rooms/36035072?adults=3&...,Το σπίτι της Zoe,2.0,4.0,1.0,1.0,"[Πιστολάκι μαλλιών, Σαμπουάν, Ζεστό νερό, Πλυν...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,38.24580,20.64420,https://www.airbnb.gr/rooms/24408776?adults=3&...,"Magnolia | Χαλαρώστε στη φύση, 8’με τα πόδια α...",3.0,6.0,1.0,2.0,"[Θέα στον κήπο, Θέα στο βουνό, Πιστολάκι μαλλι...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,38.11843,20.63374,https://www.airbnb.gr/rooms/25283091?adults=3&...,Φωτεινη studios λουρδατα,3.0,3.0,1.0,0.0,"[Πιστολάκι μαλλιών, Σαμπουάν, Ζεστό νερό, Αφρό...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,38.22857,20.41373,https://www.airbnb.gr/rooms/32358114?adults=3&...,Το σπίτι της Diana στο Seaview 1,3.0,4.0,1.0,1.0,"[Πιστολάκι μαλλιών, Ζεστό νερό, Είδη πρώτης αν...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,38.18150,20.48580,https://www.airbnb.gr/rooms/40962770?adults=3&...,Διαμέρισμα στον Κήπο του Νίκου,2.0,4.0,1.0,1.0,"[Πιστολάκι μαλλιών, Σαμπουάν, Πλυντήριο ρούχων...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
